# Libraries

In [29]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import hamming_loss

from skmultilearn.problem_transform import ClassifierChain, BinaryRelevance
from skmultilearn.adapt import MLkNN

# DataFrame

In [30]:
df_perguntas = pd.read_csv("data\stackoverflow_perguntas.csv")
df_perguntas.head()

,Perguntas,Tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js
1,"Gostaria de fazer testes unitários no Node.js,...",node.js
2,Como inverter a ordem com que o jQuery itera u...,jquery
3,Eu tenho uma página onde pretendo utilizar um ...,html
4,Como exibir os dados retornados do FireStore e...,html angular


# Investigation

In [31]:
print(f"Quantidade de perguntas: {len(df_perguntas)}")
print(f"Quantidade de tags unicas: {len(df_perguntas['Tags'].unique())}")

Quantidade de perguntas: 5408
Quantidade de tags unicas: 37


In [32]:
print(df_perguntas["Tags"].unique())

['node.js' 'jquery' 'html' 'html angular ' 'html ' 'angular' 'angular '
 'jquery html  ' 'jquery ' 'jquery html' 'jquery html ' 'html angular'
 'angular node.js ' 'html  ' 'jquery html angular' 'node.js '
 'html jquery' 'html jquery ' 'jquery angular  ' 'html node.js' 'jquery  '
 'angular node.js' 'jquery angular' 'html node.js ' 'jquery node.js '
 'angular  ' 'jquery angular ' 'jquery html angular ' 'node.js html '
 ' node.js' 'node.js html' 'html angular  ' 'jquery node.js'
 'angular html' 'html angular  node.js' 'jquery html node.js'
 'html angular node.js']


In [33]:
# criando uma lista de tags únicas a partir do dataframe de perguntas.

lista_de_tags = list()

for tags in df_perguntas["Tags"].unique():
    for tag in tags.split():
        if tag not in lista_de_tags:
            lista_de_tags.append(tag)

print(lista_de_tags)

['node.js', 'jquery', 'html', 'angular']


In [34]:
# testando para node.js

node_js = list()

for linha_tag in df_perguntas["Tags"]:
    if "node.js" in linha_tag:
        node_js.append(1)
    else:
        node_js.append(0)

df_perguntas["node.js"] = node_js

In [35]:
df_perguntas

,Perguntas,Tags,node.js
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1
2,Como inverter a ordem com que o jQuery itera u...,jquery,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0
4,Como exibir os dados retornados do FireStore e...,html angular,0
...,...,...,...
5403,Queria saber como pegar o total de cores de um...,jquery html,0
5404,"Boa noite, estou usando phonegap para fazer um...",html,0
5405,"Estou construindo um mini fórum, e nele, os us...",jquery html,0
5406,"Boa tarde, Estou para desenvolver um site na ...",html,0


In [36]:
"""
A função percorre a lista de tags e, para cada tag, verifica se ela está presente na coluna de tags do dataframe.
Se a tag estiver presente, um valor de 1 é adicionado à coluna correspondente; caso contrário, um valor de 0 é adicionado.
Ao final da execução da função, existirá uma nova coluna para cada tag na lista com os valores 0 ou 1.
"""

def nova_coluna(lista_tag, dataframe, nome_tags):
    for tag in lista_tag:
        coluna = list()
        for linha_tag in dataframe[nome_tags]:
            if tag in linha_tag:
                coluna.append(1)
            else:
                coluna.append(0)
        
        dataframe[tag] = coluna

In [37]:
nova_coluna(lista_de_tags, df_perguntas, "Tags")
df_perguntas.head()

,Perguntas,Tags,node.js,jquery,html,angular
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1


In [38]:
# condensando as tags em uma lista e adicionando-as em uma nova coluna

lista_zip_tags = list(zip(df_perguntas["node.js"], df_perguntas["jquery"], df_perguntas["html"], df_perguntas["angular"]))

df_perguntas["todas_tags"] = lista_zip_tags
df_perguntas.head()

,Perguntas,Tags,node.js,jquery,html,angular,todas_tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0,"(1, 0, 0, 0)"
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0,"(1, 0, 0, 0)"
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0,"(0, 1, 0, 0)"
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0,"(0, 0, 1, 0)"
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1,"(0, 0, 1, 1)"


# Algorithms ML

In [39]:
# separando os dados de treino e teste

perguntas_treino, perguntas_teste, tags_treino, tags_teste = train_test_split(df_perguntas["Perguntas"], df_perguntas["todas_tags"], test_size=0.2, random_state=42)

In [40]:
perguntas_treino.head()

5174    Preciso que depois que inicio a aplicação ela ...
2685    Assim como é especificado na tag p abaixo como...
1010    Estou tentando obter o valor data no campo uti...
1029    Preciso retornar um valor de CNPJ com uma másc...
4666    Estou com problemas para popular uma select co...
Name: Perguntas, dtype: object

In [41]:
"""
utilizando a TF-IDF, que é a técnica para transformar a linguagem humana em números. É uma técnica que vai dar valores diferentes para as palavras.
As que tem significância maior, um grau de distinção entre uma frase e outra, recebe um valor maior.
"""

vectorizar = TfidfVectorizer(max_features=5000, max_df=0.85)

vectorizar.fit(df_perguntas.Perguntas)
perguntas_treino_tfidf = vectorizar.transform(perguntas_treino)
perguntas_teste_tfidf = vectorizar.transform(perguntas_teste)

print(perguntas_treino_tfidf.shape)
print(perguntas_teste_tfidf.shape)

(4326, 5000)
(1082, 5000)


## One-vs-Rest strategy with Logistic Regression

In [42]:
tags_treino_array = np.asarray(list(tags_treino))
tags_teste_array = np.asarray(list(tags_teste))

In [43]:
regressao_logistica = LogisticRegression()

classificador_onevsrest = OneVsRestClassifier(regressao_logistica)
classificador_onevsrest.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_onevsrest = classificador_onevsrest.score(perguntas_teste_tfidf, tags_teste_array)
print("Resultado {0: .2f}%".format(resultado_onevsrest*100))

previsao_onevsrest = classificador_onevsrest.predict(perguntas_teste_tfidf)
hamming_loss_onevsrest = hamming_loss(tags_teste_array, previsao_onevsrest)
print("Hamming Loss {0: .2f}".format(hamming_loss_onevsrest))

Resultado  39.65%
Hamming Loss  0.19


In [44]:
df_perguntas.corr()

,node.js,jquery,html,angular
node.js,1.000000,-0.321485,-0.273523,-0.101787
jquery,-0.321485,1.000000,-0.253977,-0.366269
html,-0.273523,-0.253977,1.000000,-0.286706
angular,-0.101787,-0.366269,-0.286706,1.000000


## Classifier Chain strategy with Logistic Regression

In [45]:
classificador_cadeia = ClassifierChain(regressao_logistica)
classificador_cadeia.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_cadeia = classificador_cadeia.score(perguntas_teste_tfidf, tags_teste_array)
print("Resultado {0: .2f}%".format(resultado_cadeia*100))

previsao_cadeia = classificador_cadeia.predict(perguntas_teste_tfidf)
hamming_loss_cadeia = hamming_loss(tags_teste_array, previsao_cadeia)
print("Hamming Loss {0: .2f}".format(hamming_loss_cadeia))

Resultado  49.54%
Hamming Loss  0.21


## Binary Relevance strategy with Logistic Regression

In [46]:
classificador_br = BinaryRelevance(regressao_logistica)
classificador_br.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_br = classificador_br.score(perguntas_teste_tfidf, tags_teste_array)
print("Resultado {0: .2f}%".format(resultado_br*100))

previsao_br= classificador_br.predict(perguntas_teste_tfidf)
hamming_loss_br = hamming_loss(tags_teste_array, previsao_br)
print("Hamming Loss {0: .2f}".format(hamming_loss_br))

Resultado  39.65%
Hamming Loss  0.19


## Multilabel k-Nearest Neighbours strategy

In [47]:
classificador_mlknn = MLkNN()
classificador_mlknn.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_mlknn = classificador_mlknn.score(perguntas_teste_tfidf, tags_teste_array)
print("Resultado {0: .2f}%".format(resultado_mlknn*100))

previsao_mlknn = classificador_mlknn.predict(perguntas_teste_tfidf)
hamming_loss_mlknn = hamming_loss(tags_teste_array, previsao_mlknn)
print("Hamming Loss {0: .2f}".format(hamming_loss_mlknn))

c:\Users\vinic\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Resultado  33.27%
Hamming Loss  0.25


## Results

In [48]:
resultados_classificacao = pd.DataFrame()
resultados_classificacao["perguntas"] = perguntas_teste.values
resultados_classificacao["tags real"] = list(tags_teste)
resultados_classificacao["BR"] = list(previsao_br.toarray())
resultados_classificacao["Cadeia"] = list(previsao_cadeia.toarray())
resultados_classificacao["OneVsRest"] = list(previsao_onevsrest)
resultados_classificacao["MLkNN"] = list(previsao_mlknn.toarray())
resultados_classificacao

,perguntas,tags real,BR,Cadeia,OneVsRest,MLkNN
0,Tenho algumas inputs que deverão se autopreenc...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]","[0, 1, 1, 0]"
1,Tenho um site em PHP no qual tenho de carregar...,"(0, 1, 1, 0)","[0, 0, 0, 0]","[0.0, 0.0, 1.0, 0.0]","[0, 0, 0, 0]","[0, 0, 0, 0]"
2,Como faço para criar um popup com um CODE em ...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]","[0, 1, 0, 0]"
3,Tenho esse json CODE Preciso percorrer e tra...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]","[0, 1, 0, 0]"
4,Como faço para pegar os atributos de um objeto...,"(0, 1, 0, 0)","[0, 0, 0, 0]","[0.0, 0.0, 1.0, 0.0]","[0, 0, 0, 0]","[0, 0, 1, 0]"
...,...,...,...,...,...,...
1077,Tenho um código que forma a tela de perfil do ...,"(0, 0, 1, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]","[0, 1, 0, 0]"
1078,Como fazer para que as mensagens de validação ...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]","[0, 0, 0, 0]"
1079,"Fala galera, beleza ? Tenho uma duvida... Est...","(0, 0, 1, 0)","[0, 0, 0, 0]","[0.0, 0.0, 1.0, 0.0]","[0, 0, 0, 0]","[0, 0, 0, 0]"
1080,Hoje possuo alguns fomulários em janelas (moda...,"(0, 0, 1, 0)","[0, 1, 0, 0]","[0.0, 1.0, 0.0, 0.0]","[0, 1, 0, 0]","[0, 0, 0, 1]"


In [49]:
resultados_classificacao.perguntas.iloc[1]

'Tenho um site em PHP no qual tenho de carregar varias paginas.Porem só uma vez. O que esta acontecendo é a sobreposição de telas com varias inclusões.  CODE   Erro da Tela'